In [1]:
# install package dependencies
%pip install httpx
%pip install pandas
%pip install bs4
%pip install selenium
%pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# call all necessary imports
import httpx
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
# loading the csv files
import requests

# loading all 3 csv files
df1 = pd.read_json('./data/MSASL_test.json')
df2 = pd.read_json('./data/MSASL_train.json')
df3 = pd.read_json('./data/MSASL_val.json')


# combining all 3 files into one as we are not training a model
df = pd.concat([df1,df2,df3])
df = df.drop_duplicates(subset='clean_text',keep='first')

df.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,beer,beer,0.0,20,40,0,59,SignSchool Beer var,805,360,29.970,1.969,www.youtube.com/watch?v=wX78EPtSuzU,beer,"[0.046787232160568, 0.290409207344055, 1.0, 0....",640,NaN
1,enjoy,enjoy,0.0,11,39,0,54,enjoy(1),192,360,29.970,1.802,www.youtube.com/watch?v=OL02Odh2dRg,enjoy,"[0.071450918912887, 0.136024981737136, 1.0, 0....",480,NaN
2,emotional,emotional,0.0,20,40,0,30,SignSchool Emotional,907,360,23.976,1.251,www.youtube.com/watch?v=C59jcSo4fEI,emotional,"[0.059554219245910006, 0.281019657850265, 1.0,...",640,NaN
3,key,key,0.0,92,-1,0,37,SignSchool Key,456,360,23.976,1.543,www.youtube.com/watch?v=Qs2ua1S6tg0,key,"[0.11462894082069301, 0.308987438678741, 1.0, ...",640,NaN
4,bad,bad,0.0,11,39,0,36,BAD,81,360,29.970,1.201,www.youtube.com/watch?v=-kgTBeOw95A,bad,"[0.040453493595123007, 0.22047379612922602, 0....",480,NaN


In [4]:
# feature extraction

extracted_data = df[[
    'org_text',
    'clean_text', 
    'end_time',
    'start_time',
    'url'
]]

extracted_data.head()

,org_text,clean_text,end_time,start_time,url
0,beer,beer,1.969,0.0,www.youtube.com/watch?v=wX78EPtSuzU
1,enjoy,enjoy,1.802,0.0,www.youtube.com/watch?v=OL02Odh2dRg
2,emotional,emotional,1.251,0.0,www.youtube.com/watch?v=C59jcSo4fEI
3,key,key,1.543,0.0,www.youtube.com/watch?v=Qs2ua1S6tg0
4,bad,bad,1.201,0.0,www.youtube.com/watch?v=-kgTBeOw95A


In [15]:
# Select top 2000 most common words
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

def scrapeMostCommon():
    URL = 'https://www.talkenglish.com/vocabulary/top-2000-vocabulary.aspx'
    # service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(ChromeDriverManager().install())
    # driver = webdriver.Chrome()
    driver.get(URL)
    data = []

    soup = BeautifulSoup(driver.page_source, 'html.parser')    
    table = soup.find('table', id='GridView3')
    rows = table.find_all('tr')
    for row in rows:
        td = row.find_all('td')
        data.append(td[1].text.strip())
    return data
    
most_common_set = set(scrapeMostCommon())

C:\Users\ramas\AppData\Local\Temp\ipykernel_21356\961908856.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [16]:
# adding some more common words I will need
commonWords = [
    'bye',
]

for word in commonWords:
    most_common_set.add(word)

In [17]:
# drop words that arent in the most common word set
subset = extracted_data[extracted_data['clean_text'].isin(most_common_set)]
subset.shape


(625, 5)

In [18]:
# For progress tracking when cleaning dead links
total_urls = len(extracted_data['url'])
# 
cache = {
    
}

pattern = '"playabilityStatus":{"status":"ERROR","reason":"Video unavailable"'
def is_valid_url(url):
    if url in cache: return cache[url]
    
    try:
        
        r = requests.get(url)
        cache[url] = True
        return True
    
    except :
        # Video has been privated
        cache[url] = False
        return False

print(is_valid_url('www.youtube.com/watch?v=wX78EPtSuzU')) # should be false



UnboundLocalError: cannot access local variable 'counter' where it is not associated with a value

In [9]:
valid_rows = subset[subset['url'].apply(is_valid_url)]

0.000946969696969697 %
0.001893939393939394 %
0.002840909090909091 %
0.003787878787878788 %
0.004734848484848485 %
0.005681818181818182 %
0.006628787878787879 %
0.007575757575757576 %
0.008522727272727272 %
0.00946969696969697 %
0.010416666666666666 %
0.011363636363636364 %
0.01231060606060606 %
0.013257575757575758 %
0.014204545454545454 %
0.015151515151515152 %
0.016098484848484848 %
0.017045454545454544 %
0.017992424242424244 %
0.01893939393939394 %
0.019886363636363636 %
0.020833333333333332 %
0.021780303030303032 %
0.022727272727272728 %
0.023674242424242424 %
0.02462121212121212 %
0.02556818181818182 %
0.026515151515151516 %
0.027462121212121212 %
0.028409090909090908 %
0.029356060606060608 %
0.030303030303030304 %
0.03125 %
0.032196969696969696 %
0.03314393939393939 %
0.03409090909090909 %
0.035037878787878785 %
0.03598484848484849 %
0.036931818181818184 %
0.03787878787878788 %
0.038825757575757576 %
0.03977272727272727 %
0.04071969696969697 %
0.041666666666666664 %
0.0426136363

In [12]:
valid_rows.shape
valid_rows.to_csv('extracted_data.csv')

In [13]:
valid_rows[valid_rows['clean_text']=='bye']

,org_text,clean_text,end_time,start_time,url
